# Request API data with Python


## Index

1. Create an order
2. Monitor an order
3. Download successful captures

In [ ]:
import os
import sys
import requests
import pytz
from time import sleep
from datetime import datetime, timedelta

In [ ]:
API_KEY = "YOUR API KEY IN HERE"
PLANET_API_HOST = "api.planet.com"

### 1 | Create an order

In [ ]:
PARAMETERS = {
    'name': 'Satellite Control Demo',
    'geometry': {
        "type": "Point",
        "coordinates": [26.039818165394035, 50.524749755859375] #long, lat
    }
}

tomorrow = datetime.now(pytz.utc) + timedelta(days=1)
one_week_later = tomorrow + timedelta(days=7)

OPTIONAL_PARAMETERS = {
    'start_time': tomorrow.isoformat(),
    'end_time': one_week_later.isoformat()
}

# Optionally use optional parameters
PARAMETERS.update(OPTIONAL_PARAMETERS)

In [ ]:
import json
def create_order(parameters):

    if not API_KEY:
        print('Please set your planet api key in your environment as PLANET_API_KEY')

    headers = {
        'Authorization': f'api-key {API_KEY}',
        'Content-Type': 'application/json'
    }

    response = requests.post(f'https://{PLANET_API_HOST}/tasking/v2/orders/', json=parameters, headers=headers)

    if response.status_code == 403:
        print('Your API KEY is valid, but you are not authorized.')
    elif response.status_code == 401:
        print('Your API KEY is incorrect')
    elif response.status_code == 201:
        print('Your order was created successfully with ID: {}'.format(response.json()["id"]))
    else:
        print(f'Received status code {response.status_code} from the API. Please contact support.')

In [ ]:
create_order(PARAMETERS)

### 2 | Monitor an order

In [ ]:
def monitor_order(order_id):

    if not API_KEY:
        print('Please set your planet api key in your environment as PLANET_API_KEY')

    headers = {
        'Authorization': f'api-key {API_KEY}',
        'Content-Type': 'application/json'
    }

    response = requests.get(f'https://{PLANET_API_HOST}/tasking/v2/orders/{order_id}/', headers=headers)
    
    status = response.status_code

    if response.status_code == 403:
        print('Your API KEY is valid, but you are not authorized to view this order.')
    elif response.status_code == 401:
        print('Your API KEY is incorrect')
    elif response.status_code == 404:
        print(f'Your order ({ORDER_ID}) does not exist')
    elif response.status_code != 200:
        print(f'Received status code {response.status_code} from the API. Please contact support.')
    else:
        order = response.json()
        print(f'Your order is {order["status"]} with {order["capture_status_published_count"]} published captures '
                f'and {order["capture_assessment_success_count"]} successful captures')
        return status

In [ ]:
# Use ID of your previous order
ORDER_ID = "YOUR ORDER ID IN HERE"
monitor_order(ORDER_ID)

### 3 | Download successful captures

In [ ]:
def download_successful_captures(order_id):
    if not API_KEY:
        print('Please set your planet api key in your environment as PLANET_API_KEY')

    headers = {
        'Authorization': f'api-key {API_KEY}',
        'Content-Type': 'application/json'
    }
    
    status = monitor_order(order_id)
    
    if status == 200:
        response = requests.get(f'https://{PLANET_API_HOST}/tasking/v2/captures/?order_id={order_id}&fulfilling=true',
                                headers=headers)

        if response.status_code == 403:
            print('Your API KEY is valid, but you are not authorized to view this order.')
        elif response.status_code == 401:
            print('Your API KEY is incorrect')
        elif response.status_code != 200:
            print(f'Received status code {response.status_code} from the API. Please contact support.')
        else:
            captures = response.json()['results']
            strip_ids = [capture['strip_id'] for capture in captures]
            search_data = {
                "filter": {
                    "config": strip_ids,
                    "field_name": "strip_id",
                    "type": "StringInFilter"
                },
                "item_types": ["SkySatCollect"]
            }
            data_api_response = requests.post(f'https://{PLANET_API_HOST}/data/v1/quick-search', headers=headers,
                                              json=search_data)

            asset_urls = [feature['_links']['assets'] for feature in data_api_response.json()['features']]

            # Activate the ortho_visual asset(s)
            ortho_visual_urls = []
            for asset_url in asset_urls:
                assets = requests.get(asset_url, headers=headers).json()
                activation_url = assets['ortho_visual']['_links']['activate']
                requests.get(activation_url, headers=headers)
                ortho_visual_urls.append(assets['ortho_visual']['_links']['_self'])

            # Wait for activation and print
            for ortho_visual_url in ortho_visual_urls:
                ortho_visual = requests.get(ortho_visual_url, headers=headers).json()
                while 'location' not in ortho_visual:
                    sleep(10)
                    print('Waiting 10 seconds for asset to unlock...')
                    ortho_visual = requests.get(ortho_visual_url, headers=headers).json()
                print(f'Open the following link in a browser or download it to a file:\n{ortho_visual["location"]}')

In [ ]:
download_successful_captures(ORDER_ID)